# **POPULARITY MODEL**

### **Initial Setup**

In [2]:
!git clone https://github.com/microsoft/recommenders.git

fatal: destination path 'recommenders' already exists and is not an empty directory.


In [3]:
%cd recommenders

/content/recommenders


In [4]:
!pip install retrying

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install scrapbook

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.9 MB/s eta 0:00:00


###  **Importing the needed libraries**

In [6]:
from google.colab import drive
drive.mount('/content/drive')

import sys
import os
import numpy as np
import pandas as pd
import zipfile
from tqdm import tqdm
from tempfile import TemporaryDirectory
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.models.deeprec.deeprec_utils import download_deeprec_resources
from recommenders.models.newsrec.newsrec_utils import prepare_hparams
from recommenders.models.newsrec.models.nrms import NRMSModel
from recommenders.models.newsrec.io.mind_iterator import MINDIterator
from recommenders.models.newsrec.newsrec_utils import get_mind_data_set
from sklearn.metrics import ndcg_score
from recommenders.evaluation.python_evaluation import ndcg_at_k

import warnings
# Avoid printing some FutureWarnings
warnings.filterwarnings("ignore", category=FutureWarning)


print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))

Mounted at /content/drive
System version: 3.10.12 (main, Jun  7 2023, 12:45:35) [GCC 9.4.0]
Tensorflow version: 2.12.0


### **Loading the behavior and news dataframes**

In [7]:
# Options: demo, small, large
MIND_type = 'demo'

In [8]:
tmpdir = TemporaryDirectory()
data_path = tmpdir.name

train_news_file = os.path.join(data_path, 'train', r'news.tsv')
train_behaviors_file = os.path.join(data_path, 'train', r'behaviors.tsv')
valid_news_file = os.path.join(data_path, 'valid', r'news.tsv')
valid_behaviors_file = os.path.join(data_path, 'valid', r'behaviors.tsv')
wordEmb_file = os.path.join(data_path, "utils", "embedding.npy")
userDict_file = os.path.join(data_path, "utils", "uid2index.pkl")
wordDict_file = os.path.join(data_path, "utils", "word_dict.pkl")
yaml_file = os.path.join(data_path, "utils", r'nrms.yaml')

mind_url, mind_train_dataset, mind_dev_dataset, mind_utils = get_mind_data_set(MIND_type)

if not os.path.exists(train_news_file):
    download_deeprec_resources(mind_url, os.path.join(data_path, 'train'), mind_train_dataset)

if not os.path.exists(valid_news_file):
    download_deeprec_resources(mind_url, \
                               os.path.join(data_path, 'valid'), mind_dev_dataset)
if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.z20.web.core.windows.net/newsrec/', \
                               os.path.join(data_path, 'utils'), mind_utils)

100%|██████████| 17.0k/17.0k [00:00<00:00, 20.6kKB/s]
100%|██████████| 9.84k/9.84k [00:00<00:00, 15.6kKB/s]
100%|██████████| 95.0k/95.0k [00:01<00:00, 47.8kKB/s]


## **POPULARITY MODEL: Choosing the most viewed articles**

-------------

**Auxiliary functions**

In [9]:
def dcg_score(y_true, y_score, k=10):
    """Computing dcg score metric at k.

    Args:
        y_true (np.ndarray): Ground-truth labels.
        y_score (np.ndarray): Predicted labels.

    Returns:
        np.ndarray: dcg scores.
    """
    k = min(np.shape(y_true)[-1], k)
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])
    gains = 2 ** y_true - 1
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gains / discounts)

In [10]:
def ndcg_score(y_true, y_score, k):
    """Computing ndcg score metric at k.

    Args:
        y_true (np.ndarray): Ground-truth labels.
        y_score (np.ndarray): Predicted labels.

    Returns:
        numpy.ndarray: ndcg scores.
    """
    best = dcg_score(y_true, y_true, k)
    actual = dcg_score(y_true, y_score, k)
    return actual / best

In [11]:
def process_impression(impression_list):
    """
    Process the impression list and extract click and non-click information.

    Args:
        impression_list (str): List of impressions in string format.

    Returns:
        tuple: A tuple containing two lists - click and non-click.
    """
    list_of_strings = impression_list.split()
    click = [x.split('-')[0] for x in list_of_strings if x.split('-')[1] == '1']
    non_click = [x.split('-')[0] for x in list_of_strings if x.split('-')[1] == '0']
    return click,non_click

In [12]:
def generate_new_array(arr):
    """
    Generate a new array based on the input array, sorting the predicted news.

    Args:
        arr (list): The input array.

    Returns:
        list: The new array.
    """
    indexed_array = [(value, index) for index, value in enumerate(arr)]
    sorted_array = sorted(indexed_array, key=lambda x: x[0], reverse=True)
    new_array = [item[1] + 1 for item in sorted_array]
    return new_array

--------------

In the first cell, the 'behaviors' file is loaded into a DataFrame, `behav_df_demo`, using the `pd.read_csv` function with a tab separator. The column names are explicitly set.

A subset of this DataFrame is selected, keeping only the 'Impression_ID', 'User_ID', and 'Impressions' columns, and stored in the `behav_pop_df` DataFrame.

The 'Impressions' column in `behav_pop_df` is split into two new columns: 'Viewed Impressions' and 'Impressions array'. 'Viewed Impressions' contains the IDs of viewed items, while 'Impressions array' contains the corresponding binary values (1 for viewed, 0 for not viewed).

In [40]:
# Read the behaviors file
behav_df_demo = pd.read_csv(valid_behaviors_file,sep='\t', header=None, names=['Impression_ID', 'User_ID', 'Time', 'History', 'Impressions'])
# Select a subset
behav_pop_df = behav_df_demo.loc[:, ["Impression_ID", "User_ID", "Impressions"]]
# Split impressions
behav_pop_df["Viewed Impressions"] = behav_pop_df["Impressions"].str.split().apply(lambda x: [item.split("-")[0] for item in x if item.split("-")[1] == "1"])
behav_pop_df["Impressions labels"] = behav_df_demo["Impressions"].str.split().apply(lambda x: [int(item.split("-")[1]) for item in x])

# Display
behav_pop_df.head()

,Impression_ID,User_ID,Impressions,Viewed Impressions,Impressions labels
0,1,U41827,N23699-0 N21291-0 N1901-0 N27292-0 N17443-0 N1...,[N8620],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
1,2,U61881,N26916-0 N4641-0 N25522-0 N14893-0 N19035-0 N3...,[N19829],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,3,U54180,N13528-0 N27689-0 N10879-0 N11662-0 N14409-0 N...,[N13530],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,4,U41164,N20150-0 N1807-1 N26916-0 N28138-0 N9576-0 N19...,"[N1807, N16798]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
4,5,U8588,N21325-0 N5982-0 N19737-1 N9576-0 N20150-0 N25...,[N19737],"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"


Counting the number of times that each news article has appeared in total

In [41]:
viewed_impressions_column = behav_pop_df['Viewed Impressions']

# Split viewed impression IDs and create a list of all viewed impressions
all_viewed_impressions = [impression_id for impressions in viewed_impressions_column for impression_id in impressions]

# Count the occurrences of each viewed impression ID
viewed_impression_counts = pd.Series(all_viewed_impressions).value_counts()

# Print the viewed impression counts
viewed_impression_counts.head()

N9576     818
N26508    476
N21325    404
N20150    388
N13530    291
dtype: int64

In [42]:
behav_pop_df["Impressions array"] = behav_pop_df["Impressions"].str.split().apply(lambda x: [item.split("-")[0] for item in x])
behav_pop_df['Codes_Count'] = behav_pop_df['Impressions array'].map(lambda x: [viewed_impression_counts.get(code, 0) for code in x])
behav_pop_df['Popular prediction'] = behav_pop_df['Codes_Count'].apply(generate_new_array)

behav_pop_df.head()

,Impression_ID,User_ID,Impressions,Viewed Impressions,Impressions labels,Impressions array,Codes_Count,Popular prediction
0,1,U41827,N23699-0 N21291-0 N1901-0 N27292-0 N17443-0 N1...,[N8620],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[N23699, N21291, N1901, N27292, N17443, N18282...","[222, 2, 72, 16, 111, 99, 58, 52, 86, 9, 157, ...","[1, 21, 12, 15, 11, 20, 19, 5, 6, 28, 18, 9, 3..."
1,2,U61881,N26916-0 N4641-0 N25522-0 N14893-0 N19035-0 N3...,[N19829],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[N26916, N4641, N25522, N14893, N19035, N3877,...","[140, 67, 17, 20, 11, 26, 3, 64, 404, 282, 1, ...","[58, 20, 9, 10, 41, 47, 13, 16, 50, 1, 51, 59,..."
2,3,U54180,N13528-0 N27689-0 N10879-0 N11662-0 N14409-0 N...,[N13530],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[N13528, N27689, N10879, N11662, N14409, N6849...","[12, 77, 18, 42, 94, 58, 31, 51, 111, 17, 0, 7...","[20, 49, 36, 42, 25, 22, 9, 5, 2, 6, 21, 33, 4..."
3,4,U41164,N20150-0 N1807-1 N26916-0 N28138-0 N9576-0 N19...,"[N1807, N16798]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[N20150, N1807, N26916, N28138, N9576, N19737,...","[388, 166, 140, 157, 818, 282, 36, 28, 404, 21...","[5, 12, 9, 1, 6, 11, 10, 2, 4, 3, 7, 8, 13]"
4,5,U8588,N21325-0 N5982-0 N19737-1 N9576-0 N20150-0 N25...,[N19737],"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[N21325, N5982, N19737, N9576, N20150, N25701,...","[404, 18, 282, 818, 388, 58, 27, 42, 476, 291]","[4, 9, 1, 5, 10, 3, 6, 8, 7, 2]"


In [43]:
# We can then indexize these two new columns:
behav_pop_df['Not clicked'] = behav_pop_df['Impressions'].map(lambda x: process_impression(x)[1])
behav_pop_df["Clicks count"] = behav_pop_df["Viewed Impressions"].apply(len)

In [44]:
behav_pop_df.head()

,Impression_ID,User_ID,Impressions,Viewed Impressions,Impressions labels,Impressions array,Codes_Count,Popular prediction,Not clicked,Clicks count
0,1,U41827,N23699-0 N21291-0 N1901-0 N27292-0 N17443-0 N1...,[N8620],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[N23699, N21291, N1901, N27292, N17443, N18282...","[222, 2, 72, 16, 111, 99, 58, 52, 86, 9, 157, ...","[1, 21, 12, 15, 11, 20, 19, 5, 6, 28, 18, 9, 3...","[N23699, N21291, N1901, N27292, N17443, N18282...",1
1,2,U61881,N26916-0 N4641-0 N25522-0 N14893-0 N19035-0 N3...,[N19829],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[N26916, N4641, N25522, N14893, N19035, N3877,...","[140, 67, 17, 20, 11, 26, 3, 64, 404, 282, 1, ...","[58, 20, 9, 10, 41, 47, 13, 16, 50, 1, 51, 59,...","[N26916, N4641, N25522, N14893, N19035, N3877,...",1
2,3,U54180,N13528-0 N27689-0 N10879-0 N11662-0 N14409-0 N...,[N13530],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[N13528, N27689, N10879, N11662, N14409, N6849...","[12, 77, 18, 42, 94, 58, 31, 51, 111, 17, 0, 7...","[20, 49, 36, 42, 25, 22, 9, 5, 2, 6, 21, 33, 4...","[N13528, N27689, N10879, N11662, N14409, N6849...",1
3,4,U41164,N20150-0 N1807-1 N26916-0 N28138-0 N9576-0 N19...,"[N1807, N16798]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[N20150, N1807, N26916, N28138, N9576, N19737,...","[388, 166, 140, 157, 818, 282, 36, 28, 404, 21...","[5, 12, 9, 1, 6, 11, 10, 2, 4, 3, 7, 8, 13]","[N20150, N26916, N28138, N9576, N19737, N24553...",2
4,5,U8588,N21325-0 N5982-0 N19737-1 N9576-0 N20150-0 N25...,[N19737],"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[N21325, N5982, N19737, N9576, N20150, N25701,...","[404, 18, 282, 818, 388, 58, 27, 42, 476, 291]","[4, 9, 1, 5, 10, 3, 6, 8, 7, 2]","[N21325, N5982, N9576, N20150, N25701, N10908,...",1


Two new columns, 'Labels' and 'Sorted labels', are added to the `behav_pop_df` DataFrame.

'Labels' is constructed by concatenating a list of ones (length equal to 'Clicks count') with a list of zeros. The length of the zeros list is determined by subtracting 'Clicks count' from the length of 'Popular prediction'.

'Sorted labels' is the result of sorting 'Labels' based on the corresponding values in 'Popular prediction'. The `zip()` function pairs values from 'Popular prediction' and 'Labels', and `sorted()` orders these pairs based on the 'Popular prediction' values.


In [45]:
# Create the new array column
behav_pop_df['Labels'] = behav_pop_df.apply(lambda row: [1] * row['Clicks count'] + [0] * (len(row['Popular prediction']) - row['Clicks count']), axis=1)
behav_pop_df['Sorted labels'] = behav_pop_df.apply(lambda row: [x for _, x in sorted(zip(row['Popular prediction'], row['Labels']))], axis=1)

# Print the updated DataFrame
behav_pop_df.head()

,Impression_ID,User_ID,Impressions,Viewed Impressions,Impressions labels,Impressions array,Codes_Count,Popular prediction,Not clicked,Clicks count,Labels,Sorted labels
0,1,U41827,N23699-0 N21291-0 N1901-0 N27292-0 N17443-0 N1...,[N8620],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[N23699, N21291, N1901, N27292, N17443, N18282...","[222, 2, 72, 16, 111, 99, 58, 52, 86, 9, 157, ...","[1, 21, 12, 15, 11, 20, 19, 5, 6, 28, 18, 9, 3...","[N23699, N21291, N1901, N27292, N17443, N18282...",1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,2,U61881,N26916-0 N4641-0 N25522-0 N14893-0 N19035-0 N3...,[N19829],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[N26916, N4641, N25522, N14893, N19035, N3877,...","[140, 67, 17, 20, 11, 26, 3, 64, 404, 282, 1, ...","[58, 20, 9, 10, 41, 47, 13, 16, 50, 1, 51, 59,...","[N26916, N4641, N25522, N14893, N19035, N3877,...",1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,3,U54180,N13528-0 N27689-0 N10879-0 N11662-0 N14409-0 N...,[N13530],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[N13528, N27689, N10879, N11662, N14409, N6849...","[12, 77, 18, 42, 94, 58, 31, 51, 111, 17, 0, 7...","[20, 49, 36, 42, 25, 22, 9, 5, 2, 6, 21, 33, 4...","[N13528, N27689, N10879, N11662, N14409, N6849...",1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,4,U41164,N20150-0 N1807-1 N26916-0 N28138-0 N9576-0 N19...,"[N1807, N16798]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[N20150, N1807, N26916, N28138, N9576, N19737,...","[388, 166, 140, 157, 818, 282, 36, 28, 404, 21...","[5, 12, 9, 1, 6, 11, 10, 2, 4, 3, 7, 8, 13]","[N20150, N26916, N28138, N9576, N19737, N24553...",2,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0]"
4,5,U8588,N21325-0 N5982-0 N19737-1 N9576-0 N20150-0 N25...,[N19737],"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[N21325, N5982, N19737, N9576, N20150, N25701,...","[404, 18, 282, 818, 388, 58, 27, 42, 476, 291]","[4, 9, 1, 5, 10, 3, 6, 8, 7, 2]","[N21325, N5982, N9576, N20150, N25701, N10908,...",1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"


In [46]:
new_preds = behav_pop_df["Sorted labels"]

In [47]:
true_labels = behav_pop_df["Impressions labels"]

In [51]:
ndcg_list = [5]
for k in ndcg_list:
    ndcg_temp= np.mean(
        [
            ndcg_score(each_labels, each_preds, k)
            for each_labels, each_preds in zip(true_labels, new_preds)
        ]
    )

In [52]:
print(f'The ndcg@5 for the popularity model is {ndcg_temp}')

The ndcg@5 for the popularity model is 0.2766445014084222


In [53]:
ndcg_list = [10]
for k in ndcg_list:
    ndcg_temp= np.mean(
        [
            ndcg_score(each_labels, each_preds, k)
            for each_labels, each_preds in zip(true_labels, new_preds)
        ]
    )

In [54]:
print(f'The ndcg@10 for the popularity model is {ndcg_temp}')

The ndcg@10 for the popularity model is 0.3332973648492855


-----------

| Model   | group_auc | mean_mrr | ndcg@5 | ndcg@10 |
|----------|-----------|----------|--------|---------|
| Popularity    |   -  |   -  | 0.2766 |  0.333 |